# Meshing

## Inviscid Wedge
![Inviscid Wedge](../assets/inviscid_wedge.png)

https://su2code.github.io/tutorials/Inviscid_Wedge/

# Method 1: Manually define lines and points

In [1]:
from typing import List
from ezmesh import CurveLoop, TransfinitePlaneSurface, Geometry, Point, Line, TransfiniteLine, TransfinitePlaneSurface, visualize_mesh


with Geometry() as geo:
    mesh_size = 0.05
    points = [
        Point([0, 1], mesh_size),
        Point([1.5, 1], mesh_size),
        Point([1.5, 0.2], mesh_size),
        Point([0.5, 0], mesh_size),
        Point([0, 0], mesh_size),
    ]

    lines: List[Line] = [
        TransfiniteLine(start=points[0], end=points[1], label="upper", cell_count=150),
        TransfiniteLine(start=points[1], end=points[2], label="outlet", cell_count=200),
        TransfiniteLine(start=points[2], end=points[3], label="lower", cell_count=100),
        TransfiniteLine(start=points[3], end=points[4], label="lower", cell_count=50),
        TransfiniteLine(start=points[4], end=points[0], label="inlet", cell_count=200),
    ]

    wedge_curve_loop = CurveLoop(lines=lines)
    surface = TransfinitePlaneSurface(
        outlines=[wedge_curve_loop],
        is_quad_mesh=True,
        corners=[points[0], points[1], points[2], points[4]]
    )
    mesh = geo.generate(surface)
    visualize_mesh(mesh)
    geo.write("mesh_wedge_inv.su2")

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 20%] Meshing curve 2 (Line)
Info    : [ 40%] Meshing curve 3 (Line)
Info    : [ 60%] Meshing curve 4 (Line)
Info    : [ 80%] Meshing curve 5 (Line)
Info    : Done meshing 1D (Wall 0.000466421s, CPU 0.000652s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Transfinite)
Info    : Done meshing 2D (Wall 0.00648095s, CPU 0.0027s)
Info    : Meshing 3D...
Info    : Done meshing 3D (Wall 2.2743e-05s, CPU 1.7e-05s)
Info    : 30351 nodes 30705 elements


/usr/local/lib/python3.10/site-packages/pythreejs/traits.py:257: UserWarning: 64-bit data types not supported for WebGL data, casting to 32-bit.
  warnings.warn('64-bit data types not supported for WebGL '


HTML(value='Coords: ()')

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, far=1000.0, near=0.001, position=(0.0, 0.0, 1.0),…

,upper
,inlet
,lower
,outlet
,Zone 0


Info    : Writing 'mesh_wedge_inv.su2'...
Info    : Writing 30000 elements and 30351 nodes
Info    : Done writing 'mesh_wedge_inv.su2'


## Method 2: Automatically generate points and lines

In [2]:
from ezmesh import get_points_and_lines, CurveLoop, TransfinitePlaneSurface, Geometry, TransfinitePlaneSurface, visualize_mesh
import numpy as np

with Geometry() as geo:
    wedge_coords = np.array([[0, 1], [1.5, 1], [1.5, 0.2], [0.5, 0], [0, 0]])
    points, lines = get_points_and_lines(
        wedge_coords, 
        mesh_size = 0.05, 
        labels=["upper", "outlet", "lower", "lower", "inlet"],
        cell_counts=[150, 200, 100, 50, 200],
    )
    wedge_curve_loop = CurveLoop(lines=lines)
    surface = TransfinitePlaneSurface(
        outlines=[wedge_curve_loop],
        is_quad_mesh=True,
        corners=[points[0], points[1], points[2], points[4]]
    )
    mesh = geo.generate(surface)
    visualize_mesh(mesh)
    geo.write("mesh_wedge_inv.su2")

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 20%] Meshing curve 2 (Line)
Info    : [ 40%] Meshing curve 3 (Line)
Info    : [ 60%] Meshing curve 4 (Line)
Info    : [ 80%] Meshing curve 5 (Line)
Info    : Done meshing 1D (Wall 0.00149123s, CPU 0.000612s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Transfinite)
Info    : Done meshing 2D (Wall 0.0141931s, CPU 0.009283s)
Info    : Meshing 3D...
Info    : Done meshing 3D (Wall 2.798e-05s, CPU 2.5e-05s)
Info    : 30351 nodes 30705 elements


HTML(value='Coords: ()')

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, far=1000.0, near=0.001, position=(0.0, 0.0, 1.0),…

,upper
,inlet
,lower
,outlet
,Zone 0


Info    : Writing 'mesh_wedge_inv.su2'...
Info    : Writing 30000 elements and 30351 nodes
Info    : Done writing 'mesh_wedge_inv.su2'
